In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("data/38_feature.csv",index_col=0)
data_norm = pd.read_csv("data/38_feature_norm.csv",index_col=0)
data_std_scale = pd.read_csv("data/38_feature_std_scale.csv",index_col=0)

X = data.iloc[:,2:]
y2 = data['class2']
y4 = data['class4']
X_norm = data_norm.iloc[:,2:]
X_std_scale = data_std_scale.iloc[:,2:]

## 2-class

In [2]:
# define dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y2, stratify=y2 ,test_size=0.20, random_state=12)

# LR
from sklearn.linear_model import LogisticRegression
LR_clf =LogisticRegression(C=20,penalty='l2',solver='newton-cg',random_state=1,max_iter=1000).fit(X_train, y_train)

# SVC
from sklearn.svm import SVC
SVC_clf =SVC(C=1.0,kernel='poly',probability=True).fit(X_train, y_train)

# GB
from sklearn.ensemble import GradientBoostingClassifier
GB_clf = GradientBoostingClassifier(n_estimators = 1000,learning_rate=0.1,subsample=0.7,max_depth=7).fit(X_train, y_train)


# softvoting ensemble

from sklearn.ensemble import VotingClassifier
eclf_2 = VotingClassifier(
    estimators=[\
    ('LR', LR_clf),
    ('SVC_clf', SVC_clf),
    ('GB_clf',GB_clf)], 
    voting='soft', 
    weights=[1, 1, 1]).fit(X_train, y_train)




## 4-class

In [3]:
# define dataset
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(
#     X_norm, y4, stratify=y4 ,test_size=0.20, random_state=42)

from sklearn.svm import SVC

# Let's use all data
clf_4 =SVC(C=10,kernel='rbf',probability=True).fit(X_norm, y4)


# test_hidden data

In [4]:
X_hidden = pd.read_csv("data/npf_test_hidden.csv")
X_hidden = X_hidden[X_hidden.columns.intersection(X.columns)]
X_hidden_norm = (X_hidden - X_hidden.min())/(X_hidden.max()-X_hidden.min())
X_hidden_norm = X_hidden_norm[X_norm.columns]

In [5]:
y_4_pred = clf_4.predict(X_hidden_norm)
y_2_pred = eclf_2.predict(X_hidden_norm)
y_2_pred_prob = eclf_2.predict_proba(X_hidden_norm)

In [6]:
answer = pd.DataFrame({'class4':y_4_pred,'p':y_2_pred_prob[:,0]},)
score = 0.863353

In [7]:
answer = answer.T
answer.reset_index(inplace=True)
answer = answer.T
answer.columns = [score,""]
answer.to_csv('answer/answers.csv',index =False)